In [1]:
import os, textwrap, zipfile, json

# Root project directory
project_dir = "slope_stability_full_project"
os.makedirs(project_dir, exist_ok=True)
data_dir = os.path.join(project_dir, "data")
os.makedirs(data_dir, exist_ok=True)

# Helper function to write files
def write_file(filename, content):
    with open(os.path.join(project_dir, filename), "w", encoding="utf-8") as f:
        f.write(textwrap.dedent(content))

# ---------------- README ----------------
write_file("README.md", """
# Slope Stability Assessment in Surface Mines (2000–2025)
This project uses satellite data from Google Earth Engine to analyze slope stability in 10 open-pit mines across India.
It collects NDVI, slope, and rainfall data (2000–2025), labels it with simple rules, trains an ML model, and visualizes results in Streamlit.
""")

# ---------------- mines.json ----------------
mines = [
  {"name": "Rampura Agucha Mine", "lat": 25.8470, "lon": 74.7300},
  {"name": "Malanjkhand Copper Mine", "lat": 22.0200, "lon": 80.7050},
  {"name": "Bailadila Iron Ore Mine", "lat": 18.6500, "lon": 81.2500},
  {"name": "Khetri Copper Mine", "lat": 27.9500, "lon": 75.7830},
  {"name": "Zawar Zinc Mine", "lat": 24.3660, "lon": 73.7830},
  {"name": "Singrauli Coal Mine", "lat": 24.2000, "lon": 82.6500},
  {"name": "Dalli Rajhara Mine", "lat": 20.5830, "lon": 81.0830},
  {"name": "Neyveli Lignite Mine", "lat": 11.6100, "lon": 79.4800},
  {"name": "Rajpura Dariba Mine", "lat": 24.9660, "lon": 74.1300},
  {"name": "Joda East Iron Mine", "lat": 22.0000, "lon": 85.4000}
]
with open(os.path.join(project_dir, "mines.json"), "w") as f:
    json.dump(mines, f, indent=2)

# ---------------- collect_ee.py ----------------
write_file("collect_ee.py", """
import ee, os, json, pandas as pd

DATA_PATH = "data/features.csv"

def collect_data():
    ee.Initialize()
    with open("mines.json") as f:
        mines = json.load(f)
    years = range(2000, 2026)
    records = []

    for mine in mines:
        name = mine["name"]
        point = ee.Geometry.Point([mine["lon"], mine["lat"]])
        buffer = point.buffer(2000)

        for year in years:
            start = f"{year}-01-01"
            end = f"{year}-12-31"

            s2 = ee.ImageCollection("COPERNICUS/S2_SR")\\
                .filterDate(start, end).filterBounds(buffer)\\
                .map(lambda img: img.normalizedDifference(['B8','B4']).rename('NDVI'))

            dem = ee.Image('USGS/SRTMGL1_003')
            slope = ee.Terrain.slope(dem).rename('slope')

            rain = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY')\\
                .filterDate(start, end).filterBounds(buffer).sum().rename('rainfall')

            combined = s2.median().addBands(slope).addBands(rain)
            stats = combined.reduceRegion(reducer=ee.Reducer.mean(), geometry=buffer, scale=100, maxPixels=1e8)

            props = stats.getInfo()
            if props:
                props.update({"mine": name, "year": year})
                records.append(props)

    df = pd.DataFrame(records)
    os.makedirs("data", exist_ok=True)
    df.to_csv(DATA_PATH, index=False)
    print(f"✅ Saved dataset to {DATA_PATH}")

if __name__ == "__main__":
    ee.Authenticate()
    collect_data()
""")

# ---------------- label_data.py ----------------
write_file("label_data.py", """
import pandas as pd, os

IN_PATH = "data/features.csv"
OUT_PATH = "data/labeled_data.csv"

def rule_based_label(df):
    df['label'] = df.apply(lambda r: 1 if (r['slope'] > 25 and r['NDVI'] < 0.3 and r['rainfall'] > 500) else 0, axis=1)
    return df

def main():
    if not os.path.exists(IN_PATH):
        raise FileNotFoundError(f"{IN_PATH} not found. Run collect_ee.py first.")
    df = pd.read_csv(IN_PATH)
    df = rule_based_label(df)
    df.to_csv(OUT_PATH, index=False)
    print(f"✅ Labeled data saved to {OUT_PATH}")

if __name__ == "__main__":
    main()
""")

# ---------------- train_model.py ----------------
write_file("train_model.py", """
import pandas as pd, joblib, os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

DATA_PATH = "data/labeled_data.csv"

def train_model():
    df = pd.read_csv(DATA_PATH)
    X, y = df[['NDVI', 'slope', 'rainfall']], df['label']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    scaler = StandardScaler().fit(X_train)
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(scaler.transform(X_train), y_train)
    preds = model.predict(scaler.transform(X_test))
    print("Accuracy:", accuracy_score(y_test, preds))
    print(classification_report(y_test, preds))
    joblib.dump(model, "data/model.pkl")
    joblib.dump(scaler, "data/scaler.pkl")
    print("✅ Model and scaler saved")

if __name__ == "__main__":
    train_model()
""")

# ---------------- app.py ----------------
write_file("app.py", """
import streamlit as st, pandas as pd, json, os
from train_model import train_model
from collect_ee import collect_data

st.title("🪨 Slope Stability Dashboard (2000–2025)")

with open("mines.json") as f:
    mines = json.load(f)
mine_names = [m["name"] for m in mines]

selected_mine = st.selectbox("Select Mine", mine_names)
year_range = st.slider("Select Year Range", 2000, 2025, (2015, 2020))

data_path = "data/labeled_data.csv"
if not os.path.exists(data_path):
    st.warning("Dataset missing! Collecting data automatically...")
    collect_data()
    from label_data import main as label_main
    label_main()
    train_model()

df = pd.read_csv(data_path)
mine_df = df[df["mine"] == selected_mine]
filtered = mine_df[(mine_df["year"] >= year_range[0]) & (mine_df["year"] <= year_range[1])]

st.subheader(f"📈 Trends for {selected_mine}")
if not filtered.empty:
    st.line_chart(filtered[["year", "NDVI", "slope", "rainfall"]].set_index("year"))
else:
    st.error("No data found for this range.")
""")

# ---------------- requirements ----------------
write_file("requirements.txt", """
earthengine-api
pandas
numpy
scikit-learn
joblib
streamlit
""")

print(f"✅ Project created successfully at: {os.path.abspath(project_dir)}")
print("Run the following after activating your Python environment:")
print("1️⃣ pip install -r requirements.txt")
print("2️⃣ streamlit run app.py")


✅ Project created successfully at: C:\Users\hp\UG 2\slope_stability_full_project
Run the following after activating your Python environment:
1️⃣ pip install -r requirements.txt
2️⃣ streamlit run app.py
